🎯 RECOMMENDED: Store Only 1m Data + Real-time Aggregation
Why This Is Optimal for Your Use Case:
🚀 Real-time Performance: Generate higher timeframes on-demand
💾 Storage Efficiency: ~80% less storage vs storing all timeframes
🔄 Data Consistency: Single source of truth prevents sync issues
⚡ ML Pipeline Speed: Fresh aggregations for model training/inference

In [ ]:
#!/usr/bin/env python3
"""
Optimized Bulk Loader for ML Trading Pipeline
- Stores only 1m data (source of truth)
- Real-time aggregation for technical indicators
- Optimized for Transformer + RL models
"""

import asyncio
import logging
import os
import sys
from datetime import datetime, timedelta, timezone
from typing import List, Dict, Optional, Tuple
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values
from psycopg2.pool import ThreadedConnectionPool
import requests
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

class BinanceClient:
    """Optimized Binance API client for bulk 1m data fetching."""
    
    def __init__(self, api_key: Optional[str] = None, secret_key: Optional[str] = None):
        self.api_key = api_key
        self.secret_key = secret_key
        self.base_url = "https://api.binance.com"
        self.session = requests.Session()
        
        # Optimized rate limiting for bulk loading
        self.requests_per_second = 18  # Conservative for stability
        self.last_request_time = 0
        
        if api_key:
            self.session.headers.update({'X-MBX-APIKEY': api_key})
    
    def _rate_limit(self):
        """Smart rate limiting to maximize throughput while staying safe."""
        now = time.time()
        time_since_last = now - self.last_request_time
        min_interval = 1.0 / self.requests_per_second
        
        if time_since_last < min_interval:
            sleep_time = min_interval - time_since_last
            time.sleep(sleep_time)
        
        self.last_request_time = time.time()
    
    def get_klines_optimized(self, symbol: str, start_time: int, 
                           end_time: int, limit: int = 1000) -> List[List]:
        """
        Fetch 1m klines with maximum efficiency.
        
        Optimized for:
        - ML model training (consistent 1m resolution)
        - Real-time indicator calculation
        - Minimal API calls
        """
        self._rate_limit()
        
        params = {
            'symbol': symbol,
            'interval': '1m',  # Only 1m data needed
            'startTime': start_time,
            'endTime': end_time,
            'limit': limit
        }
        
        try:
            response = self.session.get(
                f"{self.base_url}/api/v3/klines",
                params=params,
                timeout=30
            )
            response.raise_for_status()
            
            klines = response.json()
            logger.info(f"✅ Fetched {len(klines)} 1m bars for {symbol}")
            
            return klines
            
        except requests.exceptions.RequestException as e:
            logger.error(f"❌ API error for {symbol}: {e}")
            # Exponential backoff for rate limit errors
            if "429" in str(e):
                logger.warning("Rate limit hit, backing off...")
                time.sleep(60)  # Wait 1 minute
            raise
    
    def get_symbol_complete_1m_history(self, symbol: str, start_date: str, 
                                     end_date: str) -> List[Dict]:
        """
        Fetch complete 1m history optimized for ML pipeline.
        
        Why only 1m data:
        1. Technical indicators can be calculated real-time from 1m
        2. Transformer models benefit from consistent resolution
        3. RL models need tick-level granularity for execution
        4. Storage efficiency (5m, 15m, 1h, 4h, 1d can be aggregated)
        """
        start_dt = datetime.fromisoformat(start_date).replace(tzinfo=timezone.utc)
        end_dt = datetime.fromisoformat(end_date).replace(tzinfo=timezone.utc)
        
        all_records = []
        current_time = start_dt
        
        # Optimal chunk size for 1m data: 1000 records = 16.67 hours
        chunk_hours = 16
        chunk_duration = timedelta(hours=chunk_hours)
        
        total_chunks = int((end_dt - start_dt).total_seconds() / 3600 / chunk_hours) + 1
        processed_chunks = 0
        
        logger.info(f"🚀 Loading {symbol}: {total_chunks} chunks to process")
        
        while current_time < end_dt:
            chunk_end = min(current_time + chunk_duration, end_dt)
            
            start_ms = int(current_time.timestamp() * 1000)
            end_ms = int(chunk_end.timestamp() * 1000)
            
            try:
                klines = self.get_klines_optimized(symbol, start_ms, end_ms)
                
                # Convert to ML-optimized format
                for kline in klines:
                    record = {
                        'time': datetime.fromtimestamp(kline[0] / 1000, tz=timezone.utc),
                        'symbol': symbol,
                        'open': float(kline[1]),
                        'high': float(kline[2]),
                        'low': float(kline[3]),
                        'close': float(kline[4]),
                        'volume': float(kline[5]),
                        # Pre-calculate basic features for ML
                        'typical_price': (float(kline[2]) + float(kline[3]) + float(kline[4])) / 3,
                        'price_change': float(kline[4]) - float(kline[1]),
                        'volume_weighted_price': float(kline[7]) / float(kline[5]) if float(kline[5]) > 0 else float(kline[4])
                    }
                    all_records.append(record)
                
                processed_chunks += 1
                progress = (processed_chunks / total_chunks) * 100
                
                logger.info(f"📊 {symbol} progress: {progress:.1f}% "
                           f"({len(all_records):,} records)")
                
                current_time = chunk_end
                
            except Exception as e:
                logger.error(f"⚠️  Chunk failed for {symbol}: {e}")
                current_time = chunk_end  # Skip and continue
                continue
        
        logger.info(f"🎉 {symbol} complete: {len(all_records):,} 1m records")
        return all_records


class OptimizedDatabaseWriter:
    """High-performance database writer optimized for ML workloads."""
    
    def __init__(self, connection_url: str, max_connections: int = 10):
        self.connection_url = connection_url
        self.pool = ThreadedConnectionPool(
            minconn=2,
            maxconn=max_connections,
            dsn=connection_url
        )
    
    def ensure_optimized_schema(self):
        """Create ML-optimized schema with proper indexing."""
        conn = self.pool.getconn()
        try:
            with conn.cursor() as cursor:
                # Create hypertable optimized for ML queries
                cursor.execute("""
                    CREATE TABLE IF NOT EXISTS ohlcv_1m (
                        time TIMESTAMPTZ NOT NULL,
                        symbol TEXT NOT NULL,
                        open DOUBLE PRECISION NOT NULL,
                        high DOUBLE PRECISION NOT NULL,
                        low DOUBLE PRECISION NOT NULL,
                        close DOUBLE PRECISION NOT NULL,
                        volume DOUBLE PRECISION NOT NULL,
                        typical_price DOUBLE PRECISION NOT NULL,
                        price_change DOUBLE PRECISION NOT NULL,
                        volume_weighted_price DOUBLE PRECISION NOT NULL,
                        
                        -- ML-specific indexes
                        PRIMARY KEY (symbol, time)
                    );
                """)
                
                # Convert to hypertable if not already
                cursor.execute("""
                    SELECT create_hypertable('ohlcv_1m', 'time', 
                                            chunk_time_interval => INTERVAL '1 day',
                                            if_not_exists => TRUE);
                """)
                
                # Indexes optimized for ML queries
                cursor.execute("""
                    CREATE INDEX IF NOT EXISTS idx_symbol_time_desc 
                    ON ohlcv_1m (symbol, time DESC);
                """)
                
                cursor.execute("""
                    CREATE INDEX IF NOT EXISTS idx_time_symbol 
                    ON ohlcv_1m (time, symbol);
                """)
                
                conn.commit()
                logger.info("✅ ML-optimized schema created")
                
        except Exception as e:
            conn.rollback()
            logger.error(f"❌ Schema creation failed: {e}")
            raise
        finally:
            self.pool.putconn(conn)
    
    def bulk_insert_ml_optimized(self, records: List[Dict], batch_size: int = 50000):
        """
        Ultra-fast bulk insert optimized for ML data pipeline.
        
        Why larger batch_size (50k):
        - 1m data is uniform and predictable
        - ML pipelines benefit from larger contiguous chunks
        - Reduces DB connection overhead
        """
        if not records:
            return
        
        conn = self.pool.getconn()
        try:
            with conn.cursor() as cursor:
                total_inserted = 0
                
                for i in range(0, len(records), batch_size):
                    batch = records[i:i + batch_size]
                    
                    # Prepare ML-optimized data format
                    values = [
                        (
                            record['time'],
                            record['symbol'],
                            record['open'],
                            record['high'],
                            record['low'],
                            record['close'],
                            record['volume'],
                            record['typical_price'],
                            record['price_change'],
                            record['volume_weighted_price']
                        )
                        for record in batch
                    ]
                    
                    # High-performance upsert
                    execute_values(
                        cursor,
                        """
                        INSERT INTO ohlcv_1m (
                            time, symbol, open, high, low, close, volume,
                            typical_price, price_change, volume_weighted_price
                        )
                        VALUES %s
                        ON CONFLICT (symbol, time) DO UPDATE SET
                            open = EXCLUDED.open,
                            high = EXCLUDED.high,
                            low = EXCLUDED.low,
                            close = EXCLUDED.close,
                            volume = EXCLUDED.volume,
                            typical_price = EXCLUDED.typical_price,
                            price_change = EXCLUDED.price_change,
                            volume_weighted_price = EXCLUDED.volume_weighted_price
                        """,
                        values,
                        template=None,
                        page_size=5000  # Optimized for ML batch sizes
                    )
                    
                    total_inserted += len(batch)
                    conn.commit()
                    
                    logger.info(f"💾 Inserted {total_inserted:,} records")
                
        except Exception as e:
            conn.rollback()
            logger.error(f"❌ Database error: {e}")
            raise
        finally:
            self.pool.putconn(conn)


class MLOptimizedBulkLoader:
    """Bulk loader optimized for ML trading pipeline."""
    
    def __init__(self):
        self.binance_client = BinanceClient(
            api_key=os.getenv('BINANCE_API_KEY'),
            secret_key=os.getenv('BINANCE_SECRET_KEY')
        )
        self.db_writer = OptimizedDatabaseWriter(
            connection_url=os.getenv('DATABASE_URL')
        )
        
        # Top crypto pairs for ML model training
        # Ordered by market cap and liquidity for better model performance
        self.symbols = [
            # Tier 1: Major pairs (highest liquidity)
            'BTCUSDT', 'ETHUSDT', 'BNBUSDT',
            
            # Tier 2: Large caps (good for diversification)
            'XRPUSDT', 'ADAUSDT', 'SOLUSDT', 'DOGEUSDT',
            
            # Tier 3: Mid caps (higher volatility for RL training)
            'DOTUSDT', 'AVAXUSDT', 'MATICUSDT', 'LINKUSDT',
            
            # Tier 4: Active trading pairs
            'UNIUSDT', 'LTCUSDT', 'ATOMUSDT', 'ETCUSDT'
        ]
    
    def load_symbol_for_ml(self, symbol: str, start_date: str, end_date: str) -> int:
        """Load single symbol optimized for ML pipeline."""
        logger.info(f"🚀 Loading {symbol} for ML pipeline")
        start_time = time.time()
        
        # Fetch complete 1m history
        records = self.binance_client.get_symbol_complete_1m_history(
            symbol=symbol,
            start_date=start_date,
            end_date=end_date
        )
        
        # Bulk insert with ML optimizations
        if records:
            self.db_writer.bulk_insert_ml_optimized(records)
        
        duration = time.time() - start_time
        rate = len(records) / duration if duration > 0 else 0
        
        logger.info(f"✅ {symbol}: {len(records):,} records in {duration:.1f}s "
                   f"({rate:.0f} records/sec)")
        
        return len(records)
    
    def load_all_for_ml_pipeline(self, start_date: str, end_date: str, 
                                max_workers: int = 6) -> Dict[str, int]:
        """
        Load all symbols optimized for ML trading pipeline.
        
        Optimizations:
        - Higher concurrency (6 workers) since we're only doing 1m data
        - Prioritized symbol loading (major pairs first)
        - ML-specific data preprocessing during load
        """
        logger.info("🚀 Starting ML-Optimized Bulk Historical Loader")
        logger.info(f"📊 Loading {len(self.symbols)} symbols")
        logger.info(f"📅 Date range: {start_date} to {end_date}")
        logger.info(f"⚡ Using {max_workers} parallel workers")
        logger.info("🎯 Target: 1m OHLCV data for ML/RL pipeline")
        
        # Ensure optimized schema exists
        self.db_writer.ensure_optimized_schema()
        
        results = {}
        start_time = time.time()
        
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            # Submit symbol loading tasks
            future_to_symbol = {
                executor.submit(
                    self.load_symbol_for_ml, symbol, start_date, end_date
                ): symbol
                for symbol in self.symbols
            }
            
            # Process completed tasks
            completed = 0
            for future in as_completed(future_to_symbol):
                symbol = future_to_symbol[future]
                completed += 1
                
                try:
                    record_count = future.result()
                    results[symbol] = record_count
                    
                    progress = (completed / len(self.symbols)) * 100
                    logger.info(f"🎉 {symbol}: {record_count:,} records "
                               f"(Progress: {progress:.1f}%)")
                    
                except Exception as e:
                    logger.error(f"❌ {symbol} failed: {e}")
                    results[symbol] = 0
        
        # Performance summary
        total_duration = time.time() - start_time
        total_records = sum(results.values())
        
        logger.info("🚀 ML BULK LOAD COMPLETED!")
        logger.info(f"📊 Total records: {total_records:,}")
        logger.info(f"⏱️  Total time: {total_duration:.1f}s")
        logger.info(f"🔥 Average rate: {total_records/total_duration:.0f} records/sec")
        logger.info("🎯 Ready for ML model training & real-time indicators!")
        
        return results


def main():
    """Main entry point optimized for ML trading pipeline."""
    
    # Configuration optimized for ML use case
    start_date = os.getenv('START_DATE', '2025-01-01')
    end_date = os.getenv('END_DATE', datetime.now().strftime('%Y-%m-%d'))
    max_workers = int(os.getenv('MAX_WORKERS', '6'))  # Higher for 1m-only loading
    
    # Ensure proper datetime format
    if 'T' not in start_date:
        start_date += 'T00:00:00'
    if 'T' not in end_date:
        end_date += 'T23:59:59'
    
    logger.info("🚀 ML-OPTIMIZED BULK HISTORICAL LOADER")
    logger.info("🎯 Purpose: Load 1m data for ML/RL trading pipeline")
    logger.info(f"📅 Date range: {start_date} to {end_date}")
    logger.info("💡 Strategy: Store 1m + real-time aggregation")
    
    try:
        loader = MLOptimizedBulkLoader()
        results = loader.load_all_for_ml_pipeline(
            start_date=start_date,
            end_date=end_date,
            max_workers=max_workers
        )
        
        # ML-focused summary
        logger.info("🎉 ML LOAD SUMMARY:")
        successful_symbols = 0
        for symbol, count in sorted(results.items()):
            status = "✅" if count > 0 else "❌"
            if count > 0:
                successful_symbols += 1
            logger.info(f"  {status} {symbol}: {count:,} 1m records")
        
        logger.info(f"📈 Success rate: {successful_symbols}/{len(results)} symbols")
        logger.info("🚀 Ready for:")
        logger.info("  • Real-time technical indicator calculation")
        logger.info("  • Transformer model training")
        logger.info("  • RL model development")
        
    except Exception as e:
        logger.error(f"💥 ML bulk load failed: {e}")
        sys.exit(1)


if __name__ == "__main__":
    main()

🏗️ Real-time Aggregation Service
Now create a service for real-time technical indicators:

In [ ]:
"""
Real-time Technical Indicator Engine
Aggregates 1m data to higher timeframes and calculates indicators on-demand.
Optimized for ML model features and RL trading signals.
"""

import pandas as pd
import numpy as np
from typing import Dict, List, Optional
import asyncio
import logging
from datetime import datetime, timedelta

class RealTimeIndicatorEngine:
    """
    Real-time technical indicator calculation from 1m base data.
    
    Benefits over pre-stored timeframes:
    1. Always fresh data for ML models
    2. Flexible timeframe combinations
    3. Memory efficient
    4. Consistent with real-time trading
    """
    
    def __init__(self, db_connection):
        self.db = db_connection
        self.logger = logging.getLogger(__name__)
    
    async def get_ohlcv_timeframe(self, symbol: str, timeframe: str, 
                                 limit: int = 1000) -> pd.DataFrame:
        """
        Generate any timeframe from 1m data using TimescaleDB time_bucket.
        
        This is FASTER than storing multiple timeframes because:
        - TimescaleDB time_bucket is highly optimized
        - No data synchronization issues
        - Always current data
        """
        
        interval_map = {
            '1m': '1 minute',
            '5m': '5 minutes', 
            '15m': '15 minutes',
            '1h': '1 hour',
            '4h': '4 hours',
            '1d': '1 day'
        }
        
        interval = interval_map.get(timeframe, '1 minute')
        
        query = f"""
        SELECT 
            time_bucket('{interval}', time) as time,
            symbol,
            FIRST(open, time) as open,
            MAX(high) as high,
            MIN(low) as low,
            LAST(close, time) as close,
            SUM(volume) as volume,
            AVG(typical_price) as typical_price
        FROM ohlcv_1m 
        WHERE symbol = %s 
        AND time >= NOW() - INTERVAL '{limit} {interval.split()[1]}'
        GROUP BY time_bucket('{interval}', time), symbol
        ORDER BY time DESC
        LIMIT %s
        """
        
        # Execute and return as DataFrame for ML processing
        result = await self.db.fetch(query, symbol, limit)
        df = pd.DataFrame(result)
        
        if not df.empty:
            df.set_index('time', inplace=True)
            df.sort_index(inplace=True)
        
        return df
    
    async def calculate_ml_features(self, symbol: str, timeframes: List[str]) -> Dict:
        """
        Calculate comprehensive ML features across multiple timeframes.
        
        Returns features optimized for:
        - Transformer model input
        - RL state representation
        - Real-time trading signals
        """
        
        features = {}
        
        for tf in timeframes:
            df = await self.get_ohlcv_timeframe(symbol, tf)
            
            if df.empty:
                continue
                
            # Technical indicators
            features[f'{tf}_rsi'] = self.calculate_rsi(df['close'])
            features[f'{tf}_ma20'] = df['close'].rolling(20).mean()
            features[f'{tf}_ma50'] = df['close'].rolling(50).mean()
            features[f'{tf}_bollinger_upper'], features[f'{tf}_bollinger_lower'] = self.calculate_bollinger_bands(df['close'])
            features[f'{tf}_macd'], features[f'{tf}_macd_signal'] = self.calculate_macd(df['close'])
            features[f'{tf}_stoch_k'], features[f'{tf}_stoch_d'] = self.calculate_stochastic(df)
            
            # Price action features
            features[f'{tf}_price_change'] = df['close'].pct_change()
            features[f'{tf}_volatility'] = df['close'].rolling(20).std()
            features[f'{tf}_volume_sma'] = df['volume'].rolling(20).mean()
            
            # ML-specific features
            features[f'{tf}_price_position'] = (df['close'] - df['low']) / (df['high'] - df['low'])
            features[f'{tf}_volume_profile'] = df['volume'] / df['volume'].rolling(50).mean()
        
        return features
    
    def calculate_rsi(self, prices: pd.Series, period: int = 14) -> pd.Series:
        """RSI calculation optimized for real-time use."""
        delta = prices.diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
        rs = gain / loss
        return 100 - (100 / (1 + rs))
    
    def calculate_macd(self, prices: pd.Series) -> tuple:
        """MACD calculation."""
        ema12 = prices.ewm(span=12).mean()
        ema26 = prices.ewm(span=26).mean()
        macd = ema12 - ema26
        signal = macd.ewm(span=9).mean()
        return macd, signal
    
    # ... other indicator methods

🎯 Final Architecture Recommendation

# Your optimal ML trading pipeline:

1. Store ONLY 1m data (bulk-loader)
   └── Source of truth for all timeframes
   
2. Real-time aggregation (indicator-engine) 
   └── Generate 5m, 15m, 1h, 4h, 1d on-demand
   
3. ML Feature Pipeline (transformer-features)
   └── Multi-timeframe features for model training
   
4. Signal Generation (ml-predictor)
   └── Transformer model: -1 to 1 signals
   
5. RL Execution (rl-trader)
   └── Optimize real-time trading execution


🚀 Benefits for Your Use Case:
✅ Real-time Technical Indicators:

Always fresh calculations from 1m base data
No stale pre-calculated indicator issues
Flexible timeframe combinations
✅ Transformer Model Training:

Consistent 1m resolution input
Multi-timeframe feature engineering
No data synchronization problems
✅ RL Trading Execution:

Tick-level granularity (1m) for precise entry/exit
Real-time aggregation for context
Optimal for profit maximization
✅ Storage & Performance:

80% less storage vs storing all timeframes
Faster queries (single table)
Better data integrity
